In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
from skimage.io import imread, imshow
from skimage.transform import resize
from tqdm import tqdm
import random
import tensorflow as tf
import cv2 #<- for interfacing camera and simulating functions
from IPython.display import display, clear_output
import json

from config.definitions import ROOT_DIR
from grayscaleconversionlatest import BinarizeImage

from imageprocessingLibraries import *

In [17]:
imagePath = "..\\TestImages\\english_PatientRegistrationForm_original2.jpg"

imageObj = imread(imagePath)


print("Before:")
print(imageObj.shape)

PILImshow(imageObj, imageObj)
# Image.fromarray(imageObj, mode = "RGB").save("..\\TestImages\\englishInsuranceform_lowRes.jpg")


factor = 1.5

imageObj = resize(imageObj, (imageObj.shape[0]//factor,imageObj.shape[1]//factor), mode='constant', preserve_range = True)
imageObj = imageObj.astype('uint8')
Image.fromarray(imageObj, mode = "RGB").save("..\\TestImages\\englishInsuranceform_lowRes.jpg")

print("After:")
print(imageObj.shape)

PILImshow(imageObj, imageObj)




Before:
(4032, 2859, 3)
After:
(2688, 1906, 3)


In [ ]:


testImage = ConvertToGrayScale(imread(imagePath))
testImage = BinarizeImage(testImage, 1)

In [ ]:
#Word Segmentation:

arr = np.arange(0, 100)
arr = arr.reshape(10, -1)
print(arr)

TopBoundary = 1
BottomBoundary = 3
LeftBoundary = 1
RightBoundary = 3

textlineSegment = arr[TopBoundary:BottomBoundary, LeftBoundary:RightBoundary]

In [ ]:


#Gets filenames and puts them in a list
ImagesArr = os.listdir(Image_PATH)
MasksArr = os.listdir(Masks_PATH)
MasksArr = list(MasksArr)
print(MasksArr)
print(type(MasksArr[0]))


if len(MasksArr) != len(ImagesArr):
    print("folders do not contain same number of masks and images")

X_train = np.zeros((len(MasksArr), IMG_HEIGHT, IMG_WIDTH), dtype= np.uint8)
Y_train = np.zeros((len(MasksArr), IMG_HEIGHT, IMG_WIDTH), dtype= np.uint8)

print("Loading Images...")
i = 0
for fileName in tqdm(ImagesArr):
  name = fileName.split(".")[0] # i.e convert '00000130_Text.ome.tiff' to '00000130'
  name = name + "_"
  image = imread(Image_PATH + "\\" + fileName)
  image = ConvertToGrayScale(image)
  mask = imread(Masks_PATH + "\\" + [l for l in MasksArr if name in l][0])

  X_train[i] = resize(image, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range = True)
  Y_train[i] = resize(mask, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range = True)
  i = i + 1



Imshow(image, mask)
Imshow(X_train[i-1], Y_train[i-1])

X_train = X_train/255.0
Y_train = Y_train > 127

In [ ]:
# print(X_train)
# print(Y_train)
# X_train = (X_train*255).astype(np.uint8)
# Y_train = (Y_train*255).astype(np.uint8)

i = 37
Imshow(X_train[i], Y_train[i])


In [ ]:
randInt = random.randint(0, len(X_train)-1)
# randInt = 2
print(randInt)
testImage = X_train[randInt]
testImage = testImage.reshape((1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
actualMask = Y_train[randInt]*255
predMask = model.predict(testImage)
predMask = ((predMask >= 0.7)*255).astype(np.uint8)

testImage = testImage.reshape((IMG_HEIGHT, IMG_WIDTH))
predMask = predMask.reshape((IMG_HEIGHT, IMG_WIDTH))
print(testImage.shape)
print(actualMask.shape)

Imshow((testImage*255).astype(np.uint8), actualMask)
Imshow(actualMask, predMask)
# PILImshow(actualMask, predMask)